### 1. Setup

In [18]:
import os
import json
import psycopg

json_base_path = "Final_output_files/Json_files"

print('Connecting to the local PostgreSQL database...')
conn = psycopg.connect(
    host="localhost",
    port='5432',
    dbname="my_db",
    user="postgres",
    password="123")

cur = conn.cursor()

Connecting to the local PostgreSQL database...


### 2. Creating 3 tables: categories, subtopics, articles

In [19]:
cur.execute("""
CREATE TABLE IF NOT EXISTS categories (
    id INTEGER PRIMARY KEY,
    name TEXT UNIQUE
)
""")

cur.execute("""
CREATE TABLE IF NOT EXISTS subtopics (
    id INTEGER PRIMARY KEY,
    name TEXT,
    category_id INTEGER,
    FOREIGN KEY (category_id) REFERENCES categories(id)
)
""")

cur.execute("""
CREATE TABLE IF NOT EXISTS articles (
    id SERIAL PRIMARY KEY,
    title TEXT,
    abstract TEXT,
    url TEXT,
    pub_date TEXT,
    subtopic_id INTEGER,
    FOREIGN KEY (subtopic_id) REFERENCES subtopics(id)
)
""")

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=my_db) at 0x121825610>

### 3. Insert static category and subtopic data

In [20]:
categories = {
    1: "education",
    2: "politics",
    3: "environment"
}

subtopics = {
    1: ("online learning", 1),
    2: ("higher ed", 1),
    3: ("climate policy", 3)
}

for cat_id, name in categories.items():
    cur.execute("""
        INSERT INTO categories (id, name)
        VALUES (%s, %s)
        ON CONFLICT (id) DO NOTHING
    """, (cat_id, name))

for sub_id, (name, cat_id) in subtopics.items():
    cur.execute("""
        INSERT INTO subtopics (id, name, category_id)
        VALUES (%s, %s, %s)
        ON CONFLICT (id) DO NOTHING
    """, (sub_id, name, cat_id))

### 4. Load and insert articles

In [21]:
def extract_fields(article):
    title = article.get("headline", {}).get("main", "")
    abstract = article.get("abstract", "")
    url = article.get("web_url", "")
    pub_date = article.get("pub_date", "")
    return title, abstract, url, pub_date

topic_subtopic_map = {
    "education": 2,
    "remote learning": 1,
    "online learning": 1,
    "higher ed": 2,
    "climate change": 3,
    "climate policy": 3,
    "carbon emission": 3,
    "sustainability": 3
}

for source in ["Guardian", "NewsAPI", "Nytimes"]:
    for category_id, category_name in categories.items():
        folder = os.path.join(json_base_path, source, category_name.capitalize())
        file_path = os.path.join(folder, f"{category_name}.json")

        if not os.path.exists(file_path):
            continue

        with open(file_path, "r", encoding="utf-8") as f:
            data = json.load(f)

        for article in data:
            title, abstract, url, pub_date = extract_fields(article)
            issue = article.get("issue", "").lower()
            subtopic_id = topic_subtopic_map.get(issue)

            if not subtopic_id:
                continue

            cur.execute("""
                INSERT INTO articles (title, abstract, url, pub_date, subtopic_id)
                VALUES (%s, %s, %s, %s, %s)
            """, (title, abstract, url, pub_date, subtopic_id))

conn.commit()

### Query commands

In [23]:
# Sample: view the categories table

cur.execute("SELECT * FROM categories;")
rows = cur.fetchall()

# === Print results ===
print("📋 Categories Table:")
for row in rows:
    print(row)

📋 Categories Table:
(1, 'education')
(2, 'politics')
(3, 'environment')


In [24]:
# Research question: What are the top 5 stories and total number of articles associated with Event?

top5_query = """
    SELECT title, COUNT(*) AS article_count
    FROM articles
    WHERE title IS NOT NULL AND TRIM(title) <> ''
    GROUP BY title
    ORDER BY article_count DESC
    LIMIT 5;
"""
cur.execute(top5_query)
top_5_stories = cur.fetchall()

print("🎯 Top 5 Stories:")
for i, (title, count) in enumerate(top_5_stories, 1):
    print(f"{i}. {title} ({count} articles)")

# === STEP 2: Total Articles Among Top 5 Stories ===
total_query = """
    SELECT SUM(article_count) AS total_articles
    FROM (
        SELECT COUNT(*) AS article_count
        FROM articles
        GROUP BY title
        ORDER BY article_count DESC
        LIMIT 5
    ) AS top_stories;
"""
cur.execute(total_query)
total_articles = cur.fetchone()[0]

print(f"\n📊 Total number of articles among top 5 stories: {total_articles}")

🎯 Top 5 Stories:
1. The Last 2 Months — and Next 2 Years — of U.S. Politics (6 articles)
2. Read the Letter (6 articles)
3. They Want More Babies. Now They Have Friends in the White House. (6 articles)
4. Pension Funds Push Forward on Climate Goals Despite Backlash (6 articles)
5. U.S. Faces ‘Significant Risks’ From Debt, Analysts Say, as Trump Pursues Tax Agenda (6 articles)

📊 Total number of articles among top 5 stories: 2056


In [25]:
# close the communication with the PostgreSQL
cur.close()
conn.close()